In [1]:
import tensorflow as tf
import numpy as np
import yfinance as yf
import pandas as pd

In [2]:
####constructing daily yield table for target ticker and raw pairs
class DailyPortfolio:
    def __init__(self,raw_price_table,target_ticker="TSLA"):
        self.raw_price_table = raw_price_table
        self.target_ticker = target_ticker
        self.target_history = self.get_ticker_price()
        self.price_table = self.merge_price()
        self.yield_table = self.generate_yield()
        self.spec_list = {}
    
    def get_ticker_price(self):
        target = yf.Ticker(self.target_ticker)
        target_history = target.history(period="max")
        return target_history
        
    def merge_price(self):
        price_table = pd.concat([self.raw_price_table,self.target_history['Close']],join="inner",axis=1)
        return price_table

    def generate_yield(self):
        yield_table = self.price_table/self.price_table.shift(periods=1)-1
        yield_table.replace([np.inf, -np.inf], np.nan)
        return yield_table

In [3]:
####create date-related variables#############################
####start_date=start date of the data 
####end_date=end date of the data
####port_date=start date for the portfolio prediction
####min_obs_ratio= if set 30%, raw_pair which has more than 30% of non-na yield value will be included in prediction
####set the training set date range and testing set date range -> the modle will initially be trained on x1-xt and y1-yt
####then tested by using x_{t+1} and y_{t+1}
class DateVar(DailyPortfolio):  
    def __init__(self,dailyportfol,**kwargs):

        start_date = pd.Timestamp(kwargs.get("start_date",dailyportfol.yield_table.index[1]))
        end_date = pd.Timestamp(kwargs.get("end_date",dailyportfol.yield_table.index[(dailyportfol.yield_table.shape[0]-1)]))
        dep_var = kwargs.get("dep_var",'Close')
        min_obs_ratio = kwargs.get("min_obs_ratio",0.7)        

        if "port_date" in list(kwargs.keys()):
            port_date = kwargs.get("port_date")
            port_date = pd.Timestamp(port_date)
            port_date_yesterday = pd.date_range(port_date,port_date)
            port_date_yesterday = port_date_yesterday.shift(-1,freq="D")
            port_date_yesterday = port_date_yesterday[0]
        else:
            port_date = dailyportfol.yield_table.index[round(dailyportfol.yield_table.shape[0]*0.7)]
            port_date_yesterday = port_date.shift(-1,freq="D")
            
        ini_train_index = pd.date_range(start_date,port_date_yesterday).intersection(dailyportfol.yield_table.index)
        ini_test_index = pd.date_range(port_date,end_date).intersection(dailyportfol.yield_table.index)
        
        self.ini_train_index = ini_train_index
        self.ini_test_index = ini_test_index
        self.start_date = start_date.strftime(r'%Y%m%d')
        self.end_date = end_date.strftime(r'%Y%m%d')
        self.port_date = port_date.strftime(r'%Y%m%d')


        clean_yield_table = dailyportfol.yield_table.copy()
        clean_yield_table = clean_yield_table.fillna(0)
        clean_yield_table = clean_yield_table.loc[self.ini_train_index.append(self.ini_test_index)]
        
        n_min_obs = round(clean_yield_table.shape[0]*min_obs_ratio)
        X_table = clean_yield_table.loc[:,(clean_yield_table!=0).apply(np.sum,0) > n_min_obs]
        Y_table = X_table.loc[:,[dep_var]].copy()
        X_table = X_table.drop([dep_var],axis=1)

        self.X_table = X_table
        self.Y_table = Y_table
        self.dep_var = dep_var
        self.ind_var = list(X_table.columns)
        self.min_obs_ratio = min_obs_ratio

    def generate_index(self,tt_index):
        ini_train_index = tt_index[0]
        ini_test_index = tt_index[1]
        est_index = ini_train_index
        pred_index = ini_test_index[0:1]
        left_pred_index = ini_test_index.drop(pred_index)
        ret = [(est_index, pred_index)]

        while len(left_pred_index)>0:
            est_index=est_index.append(pred_index)
            pred_index = left_pred_index[0:1]
            left_pred_index = left_pred_index.drop(pred_index)
            ret.append((est_index, pred_index))  
        return ret

In [10]:
###########fit the model with l2 norm in the objective function############
#est_data: dataset to be used for model estimation
#pred_data: dataset to be used for prediction 
#W1_B_tilde_old: parameter estimates of the previous estimation (weight of each currency in the portfolio from the previous estimation)
#lambda_l: learning_rate
#lambda_b: L2 regularization
#lambda_o: regularziation to reduce the transaction costs
class ModelFit(DateVar):
    def __init__(self,datev,**kwargs):
        
        sep_tt_index = kwargs.get("sep_tt_index",None)
        lambda_l = kwargs.get("lambda_l",0.01)
        lambda_b = kwargs.get("lambda_b",0.001)
        lambda_o = kwargs.get("lambda_o",0.001)
        epsilon = kwargs.get("epsilon",2.220446e-16)
        
        ret1 = []
        ret2 = []
        ret3 = []
        ret4 = []
        ret5 = pd.DataFrame()
        pred_index_list = []
        y_true = []

        if sep_tt_index is None:
            iterates = self.generate_index([datev.ini_train_index, datev.ini_test_index])
            port_date = datev.port_date
            ###print (port_date)
        else:
            iterates = self.generate_index(sep_tt_index)
            port_date = str(sep_tt_index[1][0].strftime(r'%Y%m%d'))

        for est_index, pred_index in iterates:
            X_est, Y_est = datev.X_table.loc[est_index], datev.Y_table.loc[est_index]
            X_pred, Y_pred = datev.X_table.loc[pred_index], datev.Y_table.loc[pred_index]
            pred_index_list.append(str(pred_index[0].strftime(r'%Y%m%d')))
            y_true.append(Y_pred.iloc[0,0])

            if port_date == str(pred_index[0].strftime(r'%Y%m%d')):
                daily_fit_result = self.fit(est_data = {"X":X_est,"Y":Y_est}, pred_data = {'X':X_pred, 'Y':Y_pred})
            else:
                daily_fit_result = self.fit(est_data = {"X":X_est,"Y":Y_est}, pred_data = {'X':X_pred, 'Y':Y_pred},W1_B_tilde_old=W1_B_tilde_old)

            ret1.append(daily_fit_result)
            ret2.append(daily_fit_result['MSE_est_ml'].iloc[0,0])
            ret3.append(daily_fit_result['MSE_pred_ml'].iloc[0,0])
            ret4.append(daily_fit_result['Y_pred_ml'].iloc[0,0])
            ret5 = pd.concat([ret5,daily_fit_result['W1_B_tilde']],axis=1)
            W1_B_tilde_old = daily_fit_result['W1_B_tilde']

        ret5.columns=pred_index_list
        ret5.index = X_est.columns

        self.all_results = ret1
        self.MSE_est = ret2
        self.MSE_pred = ret3
        self.y_pred = ret4
        self.y_true = y_true
        self.weights = ret5

    def fit(self,est_data,pred_data=None,W1_B_tilde_old=None,**kwargs):
        
        lambda_l = kwargs.get("lambda_l",0.01)
        lambda_b = kwargs.get("lambda_b",0.000001)
        lambda_o = kwargs.get("lambda_o",0.001)
        epsilon = kwargs.get("epsilon",2.220446e-16)

        if W1_B_tilde_old is None:
            initial_fit = True
            W1_B_tilde_old = 0.0
        else:
            initial_fit = False
        
        X_est = est_data['X']
        Y_est = est_data['Y']

        n_input = X_est.shape[1]

        Y_B = tf.placeholder("float")
        X_B = tf.placeholder("float", [None, n_input])
        W1_B_tilde_old_input = tf.placeholder("float")

        W1_B = tf.Variable(tf.random_normal( [n_input, 1] )) 
        W1_B_denom = tf.math.reduce_sum(tf.math.exp(W1_B))
        W1_B_tilde = tf.math.exp(W1_B)/W1_B_denom
        Y_hat = tf.matmul(X_B, W1_B_tilde)
        cost = tf.reduce_mean(tf.square(Y_hat - Y_B)) + \
               lambda_b * tf.reduce_mean(tf.square(W1_B)) + \
               tf.dtypes.cast(initial_fit==False,dtype=tf.float32) * lambda_o * tf.reduce_mean(tf.square(W1_B_tilde-W1_B_tilde_old))    

        generator = tf.train.AdamOptimizer(learning_rate = lambda_l)
        optimizer = generator.minimize(cost)

        init = tf.global_variables_initializer()
        session = tf.Session()
        session.run(init)

        
        feed_dict = {X_B:X_est, Y_B:Y_est, W1_B_tilde_old_input:W1_B_tilde_old}
        
        last_cost = np.Inf
        cost_all = None
        n_iter = 0

        cost_all = []
        while True:
            session.run(optimizer, feed_dict = feed_dict)
            current_cost = session.run(cost, feed_dict = feed_dict)
            cost_all.append(current_cost)
            if (last_cost - current_cost) < epsilon:
                break
            last_cost = current_cost
            n_iter += 1
            if n_iter > 30000:
                break
        
        Y_est_ml = session.run(Y_hat,feed_dict)
        MSE_est_ml = np.mean((Y_est_ml - Y_est)**2)##MSE for training (estimation) set

        if pred_data is None:
            Y_pred_ml = None
            MSE_pred_ml = None
        else:
            X_pred = pred_data['X']
            Y_pred = pred_data['Y']
            Y_pred_ml = session.run(Y_hat,{X_B:X_pred, Y_B:Y_pred})
            MSE_pred_ml = np.mean((Y_pred_ml - Y_pred)**2)###MSE for testing (prediction) set 
        
        t_W1_B = session.run(W1_B,feed_dict)
        t_W1_B_tilde = session.run(W1_B_tilde,feed_dict)
        
        ret = {"W1_B":pd.DataFrame(t_W1_B), "W1_B_tilde":pd.DataFrame(t_W1_B_tilde), 
        "Y_est_ml":pd.DataFrame(Y_est_ml), "MSE_est_ml":pd.DataFrame(MSE_est_ml),   
        "Y_pred_ml":pd.DataFrame(Y_pred_ml), "MSE_pred_ml":pd.DataFrame(MSE_pred_ml) 
        }

        return ret

In [9]:
class OptHyper(DateVar):
    def __init__(self,datev,**kwargs):
        
        lambda_l_list = kwargs.get("lambda_l_list",list(10.0**(-np.arange(1,3,1))))
        lambda_b_list = kwargs.get("lambda_b_list",list(10.0**(-np.arange(0,2,1))))
        lambda_o_list = kwargs.get("lambda_o_list",list(10.0**(-np.arange(0,2,1))))
        val_ratio = kwargs.get("val_ratio",0.01)

        X_train = datev.X_table.loc[datev.ini_train_index]
        Y_train = datev.Y_table.loc[datev.ini_train_index]

        hyper_param_list = [(lambda_l,lambda_b,lambda_o) for lambda_l in lambda_l_list for lambda_b in lambda_b_list for lambda_o in lambda_o_list]
        
        ######for hyperparameter optimization, initial training set is devided into 2: 99% & 1%
        start_date = datev.ini_train_index[0]
        end_date = datev.ini_train_index[-1]
        port_date = datev.ini_train_index[round(len(datev.ini_train_index)*(1-val_ratio))]
        port_date_yesterday = pd.date_range(port_date,port_date)
        port_date_yesterday = port_date_yesterday.shift(-1,freq="D")
        port_date_yesterday = port_date_yesterday[0]

        est_index = pd.date_range(start_date,port_date_yesterday).intersection(datev.ini_train_index)
        pred_index = pd.date_range(port_date,end_date).intersection(datev.ini_train_index)
        
        ret = []
        for hyper_param in hyper_param_list:
            ret.append(ModelFit(datev,sep_tt_index=[est_index, pred_index],lambda_l=hyper_param[0],lambda_b=hyper_param[1],lambda_o=hyper_param[2]))
        
        total_MSE_pred = [np.mean(x.MSE_pred) for x in ret]
        #####optimal hyper-parameter is the hyperparameter set with minimum total_MSE_pred
        opt_hyper_param = hyper_param_list[total_MSE_pred.index(min(total_MSE_pred))]
        self.all_results = ret
        self.total_MSE_pred = total_MSE_pred
        self.opt_hyper_param = opt_hyper_param
        
